# Find Clusters

In [136]:
#coloca o path raiz do projeto
working_path = "/home/thierry/Documents/thierry/"

In [137]:
import pandas as pd
import os
os.chdir(working_path)

from src.data_processment.stop_region_thierry import MovingCentroidStopRegionFinder

In [138]:
data = pd.read_csv("/home/thierry/Documents/thierry/data/user8.csv")

In [139]:
import datetime

for i in range(len(data)):
    data['date'][i] = datetime.datetime.strptime(data['date'][i], '%a %b %d %H:%M:%S GMT-03:00 %Y').day


ValueError: time data 'Wed Aug 28 13:36:54 BRT 2019' does not match format '%a %b %d %H:%M:%S GMT-03:00 %Y'

In [ ]:
data = data.query('date < 3 or date > 28')
data30s = data.iloc[ 0::2 , : ]
data60s = data.iloc[ 0::4 , : ]
data120s = data.iloc[ 0::8 , : ]

In [ ]:
data

In [ ]:
def timestamp_from_mili_to_seconds(milisseconds):
    return milisseconds / 1000

In [ ]:
#r é o raio máximo que um ponto (lat, lon) pode assumir para compor uma Stop Region, é em metros
#delta_t é o intervalo de tempo  mínimo que pracisa para formar uma Stop Region

def find_clusters(user_data, r, delta_t):
    r = 50
    delta_t = 300

    if len(user_data) == 0:
        print("User data is empty")
        return

    print("FINDING STOP REGIONS")
    stop_region_finder = MovingCentroidStopRegionFinder(region_radius=r, delta_time=delta_t)

    clusters = stop_region_finder.find_clusters(user_data, verbose=False)
    print(len(clusters), "found")

    return clusters

In [ ]:
data["local_time"] = timestamp_from_mili_to_seconds(data["timestamp"])
data30s["local_time"] = timestamp_from_mili_to_seconds(data["timestamp"])
data60s["local_time"] = timestamp_from_mili_to_seconds(data["timestamp"])
data120s["local_time"] = timestamp_from_mili_to_seconds(data["timestamp"])

In [ ]:
clusters = find_clusters(data, r=50, delta_t=5*60)
clusters30s = find_clusters(data30s, r=50, delta_t=5*60)
clusters60s = find_clusters(data60s, r=50, delta_t=5*60)
clusters120s = find_clusters(data120s, r=50, delta_t=5*60)

In [ ]:
print("Porcentagem stop regions 30s",(len(clusters30s)/len(clusters)))
print("Porcentagem stop regions 60s",(len(clusters60s)/len(clusters)))
print("Porcentagem stop regions 120s",(len(clusters120s)/len(clusters)))